In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import pickle

In [10]:
# 데이터 불러오기

# 1. 레시피, 식재료 딕셔너리 읽어오기 (100차원)
with open(r'data\recipe\recipe.pickle', 'rb') as file:
    recipe_dict = pickle.load(file)

with open(r'data\recipe\ingre.pickle', 'rb') as file:
    ingre_dict = pickle.load(file)

with open(r'data\recipe\category.pickle', 'rb') as file:
    category_dict = pickle.load(file)

rec_title = list(recipe_dict.keys()) # 키 리스트
rec_vec = [recipe_dict[rec_title[i]] for i in range(len(recipe_dict))]

# 2. 상호작용 딕셔너리 불러오기 
hos = pd.read_csv(r'data\med\disease_interaction.csv', encoding = 'cp949')
dis_dict= {hos['질병명']: {'권장영양소' : hos['권장영양소'], '주의영양소' : hos['주의영양소'], '권장식재료' : hos['권장식재료'], '주의식재료' : hos['주의식재료']} for _, hos in hos.iterrows()}

hos = pd.read_csv(r'data\med\med_interaction.csv', encoding = 'cp949')
med_dict= {hos['dname']: {'권장영양소' : hos['권장영양소'], '주의영양소' : hos['주의영양소'], '권장식재료' : hos['권장식재료'], '주의식재료' : hos['주의식재료']} for _, hos in hos.iterrows()}

In [14]:
def my_interaction(med_list, dis_list):
    interaction = {'권장식재료' : [], '주의식재료' : [], '권장영양소' : [], '주의영양소' : []}
    # 약물 정보 반영
    for med in med_list :
        for inter in interaction.keys():
            if not pd.isna(med_dict[med][inter]):
                interaction[inter].extend([value.strip() for value in med_dict[med][inter].split(',')])
    # 질병 정보 반영
    for dis in dis_list :
        for inter in interaction.keys() :
            if not pd.isna(dis_dict[dis][inter]):
                interaction[inter].extend([value.strip() for value in dis_dict[dis][inter].split(',')])
    return interaction

# my_interaction(['가나플럭스정20/1100mg', '본비바정150mg' ], ['2형 당뇨병', '뇌졸중', '구내염'])
# => 권장식재료, 주의식재료, 권장영양소, 주의영양소 리스트가 나옴

In [16]:
def recommend(my_med_list, my_dis_list, my_food):
    my_inter_dict = my_interaction(my_med_list, my_dis_list)
    my_food_vec = category_dict[my_food]
    my_food_vec_recommend = my_food_vec
    for food in my_inter_dict['권장식재료']:
        if food in ingre_dict :
            my_food_vec_recommend += 0.01* ingre_dict[food]
    for food in my_inter_dict['주의식재료']:
        if food in ingre_dict :
            my_food_vec_recommend -=  0.01 * ingre_dict[food]
    sim = cosine_similarity([my_food_vec_recommend], rec_vec)[0]
    recommend_idx = np.argsort(sim)[::-1][:5]
    return [rec_title[i] for i in recommend_idx]

In [19]:
recommend(['가나플럭스정20/1100mg', '본비바정150mg' ], ['2형 당뇨병', '뇌졸중', '구내염'], '찌개')

['해물만두칼국수 만들기 -온라인수업 아이 간단 점심',
 '감자전 만드는법 장마철 감자부침개 해먹기',
 '간식과 술안주에도 좋은 감자 해쉬브라운 만드는 법',
 '감자탕 만드는법',
 '감자 스테이크']

In [21]:
recommend(['가나플럭스정20/1100mg',], [ '뇌졸중', '구내염'], '돈까스')

['바삭바삭한 감자 고로케',
 '맑은 계란감자국 레시피',
 '감자전 비 오는날 더 맛있는 감자 스팸 전',
 '노오븐 통닭구이',
 '감자채전 만들기. 초보자도 만들수 있는 바삭한 베이컨감자전.']